In [1]:
import numpy as np
from scipy.interpolate import interp1d as interp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from src.thunderflask import thunderflask
from src.bacteria import strain
from src.ffgen import ffgen
from src.codonTable import codonTable
from src.codonUtils import utils
from src.bacteria import strain
import pickle
from src.codonOptimizer import tableOptimizer
from queue import Queue as queue
from random import choice
from matplotlib.colors import LinearSegmentedColormap as LSC
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cProfile
import copy
import random

In [2]:
#CREATES TABLES FROM 8-20 AMINO ACIDS THAT PRIORITIZE MAXIMIZING DISTANCE 
rc = utils.standardTable
#Break table into dictionaries for each amino acid
Fblock = dict((key,value) for key, value in rc.items() if value == 'F')
Lblock = dict((key,value) for key, value in rc.items() if value == 'L')
Sblock = dict((key,value) for key, value in rc.items() if value == 'S')
Yblock = dict((key,value) for key, value in rc.items() if value == 'Y')
Cblock = dict((key,value) for key, value in rc.items() if value == 'C')
Wblock = dict((key,value) for key, value in rc.items() if value == 'W')
Pblock = dict((key,value) for key, value in rc.items() if value == 'P')
Hblock = dict((key,value) for key, value in rc.items() if value == 'H')
Qblock = dict((key,value) for key, value in rc.items() if value == 'Q')
Rblock = dict((key,value) for key, value in rc.items() if value == 'R')
Iblock = dict((key,value) for key, value in rc.items() if value == 'I')
Mblock = dict((key,value) for key, value in rc.items() if value == 'M')
Tblock = dict((key,value) for key, value in rc.items() if value == 'T')
Vblock = dict((key,value) for key, value in rc.items() if value == 'V')
Nblock = dict((key,value) for key, value in rc.items() if value == 'N')
Ablock = dict((key,value) for key, value in rc.items() if value == 'A')
Dblock = dict((key,value) for key, value in rc.items() if value == 'D')
Eblock = dict((key,value) for key, value in rc.items() if value == 'E')
Gblock = dict((key,value) for key, value in rc.items() if value == 'G')
Kblock = dict((key,value) for key, value in rc.items() if value == 'K')
stopBlock = dict((key,value) for key, value in rc.items() if value == '*')
blockArray = [Fblock, Lblock, Sblock, Yblock, Cblock, Wblock, Pblock, Hblock, Qblock, Rblock,
                Iblock, Mblock, Tblock, Vblock, Nblock, Ablock, Dblock, Eblock, Gblock, Kblock, stopBlock]
        
#Iterate through all amino acid blocks selecting one key and setting the rest map to stop
firstEight = ['UUU','UCA','UAC','UGG','CUA','CCG','CAU','CGC','AUG','ACC','AAA','GUC','GCU','GAG','GGA']
choices2 = []
for i in range (len(blockArray)):
    choices2.append(random.choice(list(blockArray[i])))
    for key in blockArray[i]: 
        if key not in firstEight:
            blockArray[i][key] = '*'            
MetTable15 = {}
MetTable16 = {}
MetTable17 = {}
MetTable18 = {}
MetTable19 = {}
MetTable20 = {}
for i in range (len(blockArray)):
    MetTable15.update(blockArray[i])
MetTable16.update(MetTable15)
MetTable16['CAG'] = 'Q'
MetTable17.update(MetTable16)  
MetTable17['GAU'] = 'D'
MetTable18.update(MetTable17) 
MetTable18['AUU'] = 'I'
MetTable19.update(MetTable18) 
MetTable19['AAC'] ='N'
MetTable20.update(MetTable19)
MetTable20['UGU'] = 'C'

In [4]:
Met20Table = codonTable(MetTable20)
Met19Table = codonTable(MetTable19)
Met18Table = codonTable(MetTable18)
Met17Table = codonTable(MetTable17)
Met16Table = codonTable(MetTable16)
Met15Table = codonTable(MetTable15)

In [16]:
Met15Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : R
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : *  CGG : *
A U  AUU : *  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : *  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [17]:
Met20Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : C
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : R
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : Q  CGG : *
A U  AUU : I  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : N  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [18]:
Met19Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : R
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : Q  CGG : *
A U  AUU : I  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : N  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [19]:
Met18Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : R
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : Q  CGG : *
A U  AUU : I  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [20]:
Met17Table.dictToTable()

U        C        A        G
U U  UUU : F  UCU : *  UAU : *  UGU : *
  C  UUC : *  UCC : *  UAC : Y  UGC : *
  A  UUA : *  UCA : S  UAA : *  UGA : *
  G  UUG : *  UCG : *  UAG : *  UGG : W
C U  CUU : *  CCU : *  CAU : H  CGU : *
  C  CUC : *  CCC : *  CAC : *  CGC : R
  A  CUA : L  CCA : *  CAA : *  CGA : *
  G  CUG : *  CCG : P  CAG : Q  CGG : *
A U  AUU : *  ACU : *  AAU : *  AGU : *
  C  AUC : *  ACC : T  AAC : *  AGC : *
  A  AUA : *  ACA : *  AAA : K  AGA : *
  G  AUG : M  ACG : *  AAG : *  AGG : *
G U  GUU : *  GCU : A  GAU : D  GGU : *
  C  GUC : V  GCC : *  GAC : *  GGC : *
  A  GUA : *  GCA : *  GAA : *  GGA : G
  G  GUG : *  GCG : *  GAG : E  GGG : *

In [6]:
path = '/home/jonathan/Lab/Fast Fail/res/Reductionist Code/'
with open(path+'reductionist15.pickle', 'wb') as handle:
    pickle.dump(MetTable15, handle)